### 

In [2]:
from selenium import webdriver

In [4]:
# webdriver 경로
browser = webdriver.Chrome('chromedriver.exe')

browser.get("http://www.hanbit.co.kr/member/login.html")

# id 를 입력한다
id = browser.find_element_by_xpath('//*[@id="m_id"]')
id.send_keys('tjdwoocn')

# PWD 를 입력한다
pwd = browser.find_element_by_id('m_passwd')
pwd.send_keys('qhrtns1@')

login_submit = browser.find_element_by_class_name('btn_login')
login_submit.click()

In [24]:
from bs4 import BeautifulSoup
import time

url = 'https://play.google.com/store/apps/details?id=sixpack.sixpackabs.absworkout&showAllReviews=true'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

# 스크롤 마지막을 가져오기
last_height = driver.execute_script('return document.body.scrollHeight')

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    time.sleep(1)  # 페이지 잠시 기다리기
    
    # 새롭게 증가된 웹페이지 길이를 계산한다
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 새롭게 창을 증가시키기 위해 2400 위치까지 이동
    driver.execute_script("window.scrollTo(0,2400);")
 
    # 웹 창이 증가하지 않을 경우 나간다
    if new_height == last_height:
        break
        
    last_height = new_height
    
    
show_more = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div[2]/div/content')
show_more.click()   

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

div_add = soup.find('div', {'class':'LXrl4c'})

text_list = []
for i in div_add.findAll('span', {'jsname':'bN97Pc'}):
    text_list.append(i.get_text())
    
for k in range(len(text_list)):
    
    # 소숫점 까지 가져오기
    if str(div_add.findAll('div',{'class':'pf5lIe'})[k].div['aria-label'])[11:12]==".":
        point = float(str(div_add.findAll('div', {'class':'pf5lIe'})[k].div['aria-label'])[10:13])
    else: 
        point = float(str(div_add.findAll('div', {'class':'pf5lIe'})[k].div['aria-label'])[10:11])

    print({'점수':point, '댓글': text_list[k]})
    print()


In [26]:
show_more = driver.find_element_by_class_name('CwaK9')
show_more.click()   

### 네이버 주가 크롤링

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://finance.naver.com/sise/sise_deal_rank.nhn'

req=requests.get(url)
soup=BeautifulSoup(req.content, 'html.parser')

table_doc = soup.find('table',{'class':'type_r1'})

In [30]:
table_doc

<table cellpadding="0" cellspacing="0" class="type_r1" summary="기관 순매수 상위7 리스트">
<caption>기관 순매수 상위7</caption>
<col width="11%"/><col width="*"/><col width="30%"/><col width="13%"/>
<tr>
<td align="right" class="tr"><img alt="01" height="11" src="https://ssl.pstatic.net/imgstock/images5/ico_n01.gif" width="11"/></td>
<td><a class="company" href="/item/main.nhn?code=009830" onclick="clickcr(this, 'rdr.list', '009830', '1', event)">한화케미칼</a></td>
<td class="number">21,250</td>
<td class="tc">
<img alt="down" class="updown" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_down.gif" width="7"/>
</td>
</tr>
<tr>
<td align="right" class="tr"><img alt="01" height="11" src="https://ssl.pstatic.net/imgstock/images5/ico_n02.gif" width="11"/></td>
<td><a class="company" href="/item/main.nhn?code=122630" onclick="clickcr(this, 'rdr.list', '122630', '2', event)">KODEX 레버리지</a></td>
<td class="number">11,685</td>
<td class="tc">
<img alt="down" class="updown" height="6" src="https

In [34]:
from pandas_datareader import data
import fix_yahoo_finance as yf
yf.pdr_override()

start_date='2010-01-01'
end_data ='2019-01-10'

tickers=['005300.KS']
lotte = data.get_data_yahoo(tickers, start_date, end_data)

lotte.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,842000.0,849000.0,835000.0,841000.0,800843.750,2006
2010-01-05,841000.0,880000.0,841000.0,867000.0,825602.250,3478
2010-01-06,867000.0,867000.0,856000.0,856000.0,815127.500,1421
2010-01-07,852000.0,856000.0,851000.0,856000.0,815127.500,1328
2010-01-08,848000.0,870000.0,848000.0,870000.0,828459.125,638


### 뉴스크롤링

In [63]:
import os
import re
import time
import pickle   # 타입 변환없이 불러오는 그대로 타입지정후 사용가능토록 해줌
import requests
from bs4 import BeautifulSoup

url = 'http://find.mk.co.kr/new/search.php?pageNum=1&cat=&cat1=&media_eco=&pageSize=10&sub=all&dispFlag=OFF&page=news&s_kwd=%B7%D4%B4%EB%C4%A5%BC%BA&s_page=news&go_page=&ord=2&ord1=1&ord2=0&s_keyword=%B7%D4%B5%A5%C4%A5%BC%BA&s_i_keyword=%B7%D4%B4%EB%C4%A5%BC%BA&s_author=&y1=1991&m1=01&d1=01&y2=2018&m2=12&d2=26&ord=2&area=ttbd'

# url requests
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

# 뉴스 개수 파악
news_count = re.findall('\d+', soup.find('div',{'sub_top'}).text)[:-2]

# 뉴스 개수를 계산하여 page number 변수를 선언
news_count = int(news_count[0] + news_count[1])
page_number = 0
if news_count%20 !=0:
    page_number = int(news_count/20)+1
else:
    page_number = int(news_count/20)
    
# make parse
time.sleep(5)

news_index = 0

# each page crawling
# for date_page in range(1,10):
for date_page in range(1000, 1010):
    print('page number: ', date_page, '/', page_number)
    
    url = 'http://find.mk.co.kr/new/search.php?pageNum='+str(date_page)+'&cat=&cat1=&media_eco=&pageSize=10&sub=all&dispFlag=OFF&page=news&s_kwd=%B7%D4%B4%EB%C4%A5%BC%BA&s_page=news&go_page=&ord=2&ord1=1&ord2=0&s_keyword=%B7%D4%B5%A5%C4%A5%BC%BA&s_i_keyword=%B7%D4%B4%EB%C4%A5%BC%BA&s_author=&y1=1991&m1=01&d1=01&y2=2018&m2=12&d2=26&ord=2&area=ttbd'

    group = '롯데칠성'
    
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    news_list = soup.find_all('div', {'class': 'sub_list'})  
    news_ = {}
    
    # make news link
    for i in news_list:
        date_news_list = re.findall('\d+',i.find('span',{'class':'art_time'}).text)
        
    # index date time preprocessing  , 뉴스별로 올리는 시간이 초단위로 보면 다를것이라 보고, 시간을 인덱스로 사용하기 위해 이렇게 함
    date_index = str(date_news_list[0]+'.'+str(date_news_list[1])+'.'+str(date_news_list[2])+'.'
                     +str(date_news_list[3])+'.'+str(date_news_list[4])+'.'+str(date_news_list[5]))  
    
    # total date make dict type, 딕셔너리 형식으로 저장하게 만들었음
    news_[date_index] = {'date': date_index,
                        'link': i.a['href'],
                        'title':i.find('span',{'class': 'art_tit'}).text,
                         'group' : group}
    news_index+=1
        
    # news_crawling
    for nl in list(news_.keys())[:]:
        link_url = news_[nl]['link']
        link_req = requests.get(link_url)
        
        time.sleep(5)
        
        # if status code 200, do code , 인터넷에 접속했을시 접속이 잘됫으면 200, 잘못됬으면 다른 값을 보내줌
        if link_req.status_code ==200:
            link_soup = BeautifulSoup(link_req.content, 'html.parser')
            try:
                script = link_soup.find('div', {'class':'art_txt'}).text
            except Exception as e:
                print(e)
                pass
            news_[nl].update({'script':script})   # script 부분에 새로운 script 값을 업데이트 해주겟다
        
        #save crawling data
        with open('./텍스트마이닝/'+group+'/news_' +str(date_page)+'.pickle', 'wb') as f:
            pickle.dump(news_, f, pickle.HIGHEST_PROTOCOL)   # 피클을 써줄때는 .dump를 해줌
            print('done')
        

page number:  1000 / 1325
done
page number:  1001 / 1325
done
page number:  1002 / 1325
done
page number:  1003 / 1325
done
page number:  1004 / 1325
done
page number:  1005 / 1325
done
page number:  1006 / 1325
done
page number:  1007 / 1325
done
page number:  1008 / 1325
done
page number:  1009 / 1325
done


### 뉴스 전처리

In [83]:
import os
import math
import pickle
import pandas as pd
from datetime import datetime
from collections import Counter
from konlpy.tag import Twitter
tw = Twitter()

#data load 0114.crawling_1에서 만든 피클 파일 전부를 불러온다.
pic_list = []
for (path, dir_, files) in os.walk('./텍스트마이닝/롯데칠성'):
    if files != []:
        for pic in files:
            pic_list.append('./텍스트마이닝/롯데칠성/'+pic[:])
            #pic_list: ['C:/Users/user/AI_Python_Study/매일경제/롯데칠성/news_1.pickle',
#  'C:/Users/user/AI_Python_Study/매일경제/롯데칠성/news_2.pickle']

news_ = {}
for i in pic_list[:]:
    with open(i, 'rb') as f:
        news_.update(pickle.load(f)) # 모든 피클 파일을 news_에 집어넣는다.

index_date = [i.strftime('%Y-%m-%d') for i in pd.date_range('20130110', '20130201')]
# date_range에 해당되는 모든 날에 대해 일치하는 날짜가 있으면 분석해서 데이터를 넣는다
news_each_day = {} # make news each day

for idx in index_date[:]:
    news_date = ''
    cnt=1
    for find_date in news_.keys():
        if news_[find_date]['date'][:10].replace('.','-') == idx: # 인덱스와 news 날짜가 일치하면
            news_date += str(news_[find_date]['script'])
            print(find_date,news_date)
            cnt += 1
    news_each_day[idx] = {'date':idx, 'script':news_date}
    print(news_each_day[idx])

token_ = {}
for i in list(news_each_day.keys())[:]:
#     if news_each_day[i]['script'] != '': 원래 코드에는 이게 있는데 이걸 쓰면 뭔가 큰일이 일어난다
    token_[i] = {'date':datetime.strptime(news_each_day[i]['date'], '%Y-%m-%d'), # string을 datetime으로 변경
                'nouns':tw.nouns(news_each_day[i]['script'])}
    
group='형태소'
with open('./텍스트마이닝/'+group+'/news_.pickle', 'wb') as f:
    pickle.dump(token_, f, pickle.HIGHEST_PROTOCOL)
   
   
#data load
pic_list = []
for (path, dir_, files) in os.walk('./텍스트마이닝/형태소/'):
    if files != []:
        for pic in files:
            pic_list.append('./텍스트마이닝/형태소/'+pic[:])
news_ = {}
for i in pic_list[:]:
    with open(i, 'rb') as f:
        news_.update(pickle.load(f))
# print(news_)

{'date': '2013-01-10', 'script': ''}
{'date': '2013-01-11', 'script': ''}
{'date': '2013-01-12', 'script': ''}
{'date': '2013-01-13', 'script': ''}
{'date': '2013-01-14', 'script': ''}
{'date': '2013-01-15', 'script': ''}
{'date': '2013-01-16', 'script': ''}
{'date': '2013-01-17', 'script': ''}
{'date': '2013-01-18', 'script': ''}
{'date': '2013-01-19', 'script': ''}
{'date': '2013-01-20', 'script': ''}
{'date': '2013-01-21', 'script': ''}
2013.01.22.13.46.54 3년째 포화상태…동서·남양·네슬레 순으로 시장 점유농심 신규진출…시장 안착할지 미지수
커피믹스 시장이 올해도 뜨겁게 끓어오를  전망이다. 
22일 식품업계에 따르면 지난해 커피믹스 시장은 1조2천억원 규모로 추산돼  최근 3개년째 비슷한 수준을 기록, 포화상태인 것으로 나타났다.
이날 AC닐슨이 발표한 지난해 전국 커피믹스 시장 업체별 점유율은 동서가 79.6%로 1위를 차지했다. 
남양유업이 12.6%를 차지하며 2위에 올랐으며 네슬레(5.1%)와 롯데칠성음료(1.4%)가 각각 3위와 4위를 기록했다.googletag.display("google_dfp_MC_2x1,fluid");     애초 동서가 시장의 90%를, 네슬레가 10%를 양분했던 커피믹스 시장은 남양이 2010년 12월 `프렌치카페`를 내놓으면서 판도가 바뀌었다. 
남양 프렌치카페는 출시 6개월만에 네슬레를 제치고 두자릿수 시장 점유율을 기록했다. 
이어 타 업체들이 속속 커피믹스 시장에 뛰어들면서 경쟁이 점차 가열되고 있다. 
롯데칠성음료가 지난해 6월 `칸타타`를 출시한 데 

In [84]:
pic_list = []
for (path, dir_, files) in os.walk('./텍스트마이닝/형태소//'):
    if files != []:
        for pic in files:
            pic_list.append('./텍스트마이닝/형태소//' + pic[:])
            
news_ = {}
for i in pic_list[:]:
    #load
    with open(i, 'rb') as f:
        news_.update(pickle.load(f))
        print(news_)

{'2013-01-24': {'date': datetime.datetime(2013, 1, 24, 0, 0), 'nouns': []}, '2013-01-26': {'date': datetime.datetime(2013, 1, 26, 0, 0), 'nouns': []}, '2013-01-13': {'date': datetime.datetime(2013, 1, 13, 0, 0), 'nouns': []}, '2013-01-18': {'date': datetime.datetime(2013, 1, 18, 0, 0), 'nouns': []}, '2013-01-21': {'date': datetime.datetime(2013, 1, 21, 0, 0), 'nouns': []}, '2013-01-16': {'date': datetime.datetime(2013, 1, 16, 0, 0), 'nouns': []}, '2013-01-15': {'date': datetime.datetime(2013, 1, 15, 0, 0), 'nouns': []}, '2013-01-25': {'date': datetime.datetime(2013, 1, 25, 0, 0), 'nouns': []}, '2013-01-30': {'date': datetime.datetime(2013, 1, 30, 0, 0), 'nouns': []}, '2013-01-10': {'date': datetime.datetime(2013, 1, 10, 0, 0), 'nouns': []}, '2013-01-17': {'date': datetime.datetime(2013, 1, 17, 0, 0), 'nouns': []}, '2013-01-20': {'date': datetime.datetime(2013, 1, 20, 0, 0), 'nouns': []}, '2013-02-01': {'date': datetime.datetime(2013, 2, 1, 0, 0), 'nouns': []}, '2013-01-23': {'date': da

### 감성사전로드

In [85]:
sentiment_dict_df=pd.read_csv('텍스트마이닝/lexicon/polarity.csv',encoding='utf-8')
def dictionary_prepro(sentiment_dict_df):
    Score_df = pd.DataFrame()
    
    #감성사전 분할 및 전처리.
    for sent_df_row in range(len(sentiment_dict_df)):
        sentiment_dict_df['ngram'][sent_df_row].split(';')
        for find_noun_row in range(len(sentiment_dict_df['ngram'][sent_df_row].split(';'))):
            if sentiment_dict_df['ngram'][sent_df_row].split(';')[find_noun_row].split('/')[-1] == 'NNG':
    #ngram 값이 NNG일 경우 Score_df추가
                Score_df=pd.concat([Score_df,sentiment_dict_df.iloc[sent_df_row,:]], axis=1)
    #형용사가 없는 row일 경우 
            if sentiment_dict_df['ngram'][sent_df_row].split('/')[-1]=='NNG':
                Score_df=pd.concat([Score_df,sentiment_dict_df.iloc[sent_df_row,:]], axis=1)
    Score_df = Score_df.T.reset_index()
    return Score_df

Score_df = dictionary_prepro(sentiment_dict_df)

remove_same = []
for i in range(len(Score_df['ngram'])):
    for j in range(len(Score_df['ngram'][i])):
        try: 
            remove_same.append(Score_df['ngram'][i].split(';')[j].split('/')[0]+'/'+str(Score_df['POS'][i] - Score_df['NEG'][i])) 
        except Exception as e:
            
            #print(e)
            
            remove_same.append(Score_df['ngram'][i].split('/')[0]+'/'+str(Score_df['POS'][i]-Score_df['NEG'][i]))
            
set_values = {}
for set_div in list(set(remove_same)):
    div_words = set_div.split('/')
    set_values[div_words[0]] = float(div_words[1])

C:\Users\tjdwo\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  
C:\Users\tjdwo\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # This is added back by InteractiveShellApp.init_path()


In [86]:
set_values

{'공식': -1.0,
 '지속': -1.0,
 '비슷': 1.0,
 '떠오르': 1.0,
 '풍물': 1.0,
 '석가모니': 0.0,
 '쿠데타': -1.0,
 '호감': 1.0,
 '센스': 1.0,
 '독자': -1.0,
 '표상': -1.0,
 '뜨': 0.0,
 '흘리': -1.0,
 '싫증': -1.0,
 '망령': -1.0,
 '반발': -1.0,
 '건축': 0.0,
 '독서': 1.0,
 '제품': 0.333333333,
 '쓸쓸': 0.0,
 '극찬': 1.0,
 '지위': 1.0,
 '감동': 1.0,
 '끼': 1.0,
 '위': -1.0,
 '납득': -1.0,
 '이름': 0.0,
 '높': 0.0,
 '레크리에이션': 1.0,
 '고육지책': -1.0,
 '묻': -1.0,
 '존재': 1.0,
 '불의': -1.0,
 '우연': -1.0,
 '등산인': 1.0,
 '칼날': -1.0,
 '장래': 1.0,
 '아내': -1.0,
 '울리': 0.0,
 '결의': 1.0,
 '상품': -1.0,
 '제시': 0.75,
 '청구': -1.0,
 '면모': 1.0,
 '생기': 1.0,
 '프로': 1.0,
 '급': 1.0,
 '공존': 1.0,
 '수분': -1.0,
 '비리': -1.0,
 '지혜': 1.0,
 '유치원': 1.0,
 '연구': 0.0,
 '부부': -1.0,
 '올리': -1.0,
 '그리하여': -1.0,
 '고이': -1.0,
 '허둥대': -1.0,
 '지난번': 1.0,
 '아파트': -1.0,
 '풀타임': 1.0,
 '벌어지': 1.0,
 '비유': 0.0,
 '다투': 1.0,
 '공유': 1.0,
 '예술품': -1.0,
 '풀': 1.0,
 '근본': 1.0,
 '현란*': -1.0,
 '하': 0.333333334,
 '독특': 1.0,
 '스타': 1.0,
 '몰리': 1.0,
 '성공': 0.8,
 '죄의식': -1.0,
 '극대': -1.0,
 '그리': -1.0,
 '효도': 1.0,
 

In [90]:
with open('./텍스트마이닝/형태소/news_.pickle', 'rb') as f:
    data = pickle.load(f)

# load tokenize analysis data
add_nouns_each_day = {}
for i in list(data.keys())[:]:
    if data[i]['nouns'] != []:
        add_nouns_each_day[i] = []
# print(len(data['1991-01-05']['nouns']))

# 단음절 처리
for i in list(data.keys())[:]:
    if data[i]['nouns'] != []:
        word_len = []
        for j in data[i]['nouns']:
            if len(j) > 0: # () 안에 들어갈 숫자는?
                word_len.append(j)
        add_nouns_each_day[i] += word_len
# make word frequency
word_frequency = {}
for i in list(add_nouns_each_day.keys())[:]:
    count_ = Counter(add_nouns_each_day[i])
    word_frequency[i] = dict(count_)
    
# 단어 출현 횟수 * 사전 점수
frequency_sentiment_point = {}
for i in list(word_frequency.keys())[:]:
    intersection_word = set(word_frequency[i]) & set(set_values) # 교집합! 사전에도 있고 뉴스에도 있는 명사를 가져온다
    day_point = 0
    for intword in intersection_word:
        day_point += word_frequency[i][intword] * set_values[intword]
        
        frequency_sentiment_point[i] = {'point': day_point}
with open('./텍스트마이닝/형태소/_test_.pickle', 'wb') as f:
    pickle.dump(frequency_sentiment_point, f, pickle.HIGHEST_PROTOCOL)

#data load
pic_list = []
for (path, dir_, files) in os.walk('./텍스트마이닝/형태소/'):
    if files != []:
        for pic in files:
            pic_list.append('./텍스트마이닝/형태소/'+pic[:])
            
news_ = {}

i = './텍스트마이닝//형태소/_test_.pickle'
with open(i, 'rb') as f:
    news_.update(pickle.load(f))
print(news_)

{'2013-01-28': {'point': 44.748717942000006}, '2013-01-22': {'point': 3.919047627000001}}


In [91]:
news_

{'2013-01-22': {'point': 3.919047627000001},
 '2013-01-28': {'point': 44.748717942000006}}